# Start analysis

## Import

In [1]:
from pathlib import Path
import os
import csv
import pandas as pd
import datetime
import sqlite3
import logging


## Locate path 

In [2]:
# Locate the CSV under a relative data folder: try ./data then ../data (if the notebook runs inside ./notebook).
cwd = Path.cwd()
data_dir_candidates = [cwd / "data", cwd.parent / "data"]  # supports both project root and notebook/ as CWD
data_dir = next((p for p in data_dir_candidates if p.exists()), data_dir_candidates[0])
data_dir.mkdir(parents=True, exist_ok=True)

expected_csv_name = "Retail_supply_chain - Retails Order Full Dataset.csv"
csv_path = data_dir / expected_csv_name

if csv_path.exists():
    print(f"OK: CSV found -> {csv_path}")
else:
    # Fallback: try to find a similar CSV recursively under the chosen data_dir
    candidates = list(data_dir.rglob("Retail_supply_chain*Full*Dataset*.csv"))
    if candidates:
        csv_path = candidates[0]
        print(f"Notice: expected file not found, using -> {csv_path}")
    else:
        raise FileNotFoundError(
            f"CSV not found. Expected at: {csv_path}\n"
            f"Place the file in the 'data/' folder at project root."
        )



OK: CSV found -> c:\Users\pc\Desktop\PROJECTS\Supply_chain_analysis\data\Retail_supply_chain - Retails Order Full Dataset.csv


In [3]:
# Create an empty SQLite database file under a relative data folder (./data or ../data).

cwd = Path.cwd()
data_dir_candidates = [cwd / "data", cwd.parent / "data"]
data_dir = next((p for p in data_dir_candidates if p.exists()), data_dir_candidates[0])
data_dir.mkdir(parents=True, exist_ok=True)

db_path = data_dir / "Retail_supply_chain.db"
conn = sqlite3.connect(db_path.as_posix())
conn.close()

print(f"OK: DB created -> {db_path}  | Exists: {db_path.exists()}")


OK: DB created -> c:\Users\pc\Desktop\PROJECTS\Supply_chain_analysis\data\Retail_supply_chain.db  | Exists: True


In [4]:
# Helpers
def to_iso_date(s: str):
    if s is None:
        return None
    s = str(s).strip()
    if not s:
        return None
    for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%d-%m-%Y", "%m-%d-%Y"):
        try:
            return datetime.datetime.strptime(s, fmt).date().isoformat()
        except Exception:
            continue
    return s  # keep as-is if already ISO or unparseable but non-empty

def parse_float(s):
    if s is None:
        return None
    s = str(s).strip()
    if not s:
        return None
    # Remove currency/percent and spaces (incl. non-breaking)
    for ch in ["€", "%", "\u00A0", " "]:
        s = s.replace(ch, "")
    # Normalize thousands/decimal separators
    if "," in s and "." in s:
        # Assume '.' as thousands and ',' as decimal (e.g., 1.234,56)
        s = s.replace(".", "").replace(",", ".")
    elif "," in s:
        # Assume ',' is decimal (e.g., 261,96)
        s = s.replace(",", ".")
    try:
        return float(s)
    except Exception:
        return None  # fallback: treat as missing

def parse_int(s):
    # Parse integers reliably even if given like "1.000" or "1,000"
    f = parse_float(s)
    if f is None:
        return None
    try:
        return int(round(f))
    except Exception:
        return None

# Paths (relative-safe: supports running from project root or ./notebook)
cwd = Path.cwd()
data_dir_candidates = [cwd / "data", cwd.parent / "data"]
data_dir = next((p for p in data_dir_candidates if p.exists()), data_dir_candidates[0])
db_path = data_dir / "Retail_supply_chain.db"
expected_csv_name = "Retail_supply_chain - Retails Order Full Dataset.csv"
csv_path = data_dir / expected_csv_name
if not csv_path.exists():
    candidates = list(data_dir.rglob("Retail_supply_chain*Full*Dataset*.csv"))
    if not candidates:
        raise FileNotFoundError(f"CSV not found at {csv_path}; place the file in the 'data/' folder.")
    csv_path = candidates[0]

# Schema
schema_sql = """
DROP TABLE IF EXISTS stg_orders;
CREATE TABLE stg_orders (
    row_id               INTEGER,
    order_id             TEXT,
    order_date           TEXT,   -- ISO 8601 YYYY-MM-DD
    ship_date            TEXT,   -- ISO 8601 YYYY-MM-DD
    ship_mode            TEXT,
    customer_id          TEXT,
    customer_name        TEXT,
    segment              TEXT,
    country              TEXT,
    city                 TEXT,
    state                TEXT,
    postal_code          INTEGER,
    region               TEXT,
    retail_sales_people  TEXT,
    product_id           TEXT,
    category             TEXT,
    sub_category         TEXT,
    product_name         TEXT,
    returned             TEXT,
    sales                REAL,
    quantity             INTEGER,
    discount             REAL,
    profit               REAL
);
CREATE INDEX IF NOT EXISTS idx_stg_orders_order_date ON stg_orders(order_date);
CREATE INDEX IF NOT EXISTS idx_stg_orders_state ON stg_orders(state);
CREATE INDEX IF NOT EXISTS idx_stg_orders_region ON stg_orders(region);
CREATE INDEX IF NOT EXISTS idx_stg_orders_category ON stg_orders(category, sub_category);
"""

# Load
conn = sqlite3.connect(db_path.as_posix())
cur = conn.cursor()
cur.executescript(schema_sql)
conn.commit()

with open(csv_path, "r", encoding="utf-8-sig", newline="") as f:
    reader = csv.DictReader(f)
    rows = []
    for r in reader:
        rows.append((
            parse_int(r.get("Row ID")),
            (r.get("Order ID") or "").strip() or None,
            to_iso_date(r.get("Order Date")),
            to_iso_date(r.get("Ship Date")),
            (r.get("Ship Mode") or "").strip() or None,
            (r.get("Customer ID") or "").strip() or None,
            (r.get("Customer Name") or "").strip() or None,
            (r.get("Segment") or "").strip() or None,
            (r.get("Country") or "").strip() or None,
            (r.get("City") or "").strip() or None,
            (r.get("State") or "").strip() or None,
            parse_int(r.get("Postal Code")),
            (r.get("Region") or "").strip() or None,
            (r.get("Retail Sales People") or "").strip() or None,
            (r.get("Product ID") or "").strip() or None,
            (r.get("Category") or "").strip() or None,
            (r.get("Sub-Category") or "").strip() or None,
            (r.get("Product Name") or "").strip() or None,
            (r.get("Returned") or "").strip() or None,
            parse_float(r.get("Sales")),
            parse_int(r.get("Quantity")),
            parse_float(r.get("Discount")),
            parse_float(r.get("Profit"))
        ))

insert_sql = """
INSERT INTO stg_orders (
    row_id, order_id, order_date, ship_date, ship_mode, customer_id, customer_name,
    segment, country, city, state, postal_code, region, retail_sales_people,
    product_id, category, sub_category, product_name, returned, sales, quantity, discount, profit
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""
cur.executemany(insert_sql, rows)
conn.commit()

total_rows = cur.execute("SELECT COUNT(*) FROM stg_orders;").fetchone()[0]
print(f"OK: loaded {total_rows} rows into stg_orders")

conn.close()



OK: loaded 9994 rows into stg_orders


In [5]:
# Phase 2: build Rsc_cleaned from stg_orders (no new files; aligns with your last setup)

# Resolve DB path (supports running from project root or ./notebook)
cwd = Path.cwd()
data_dir_candidates = [cwd / "data", cwd.parent / "data"]
data_dir = next((p for p in data_dir_candidates if p.exists()), data_dir_candidates[0])
db_path = data_dir / "Retail_supply_chain.db"

# Cleaning / standardization SQL
cleaning_sql = """
-- 1) Deduplicate on row_id (keep most recent by order_date; fallback by rowid)
DROP TABLE IF EXISTS stg_orders_dedup;
CREATE TABLE stg_orders_dedup AS
WITH ranked AS (
  SELECT
    s.*,
    ROW_NUMBER() OVER (
      PARTITION BY s.row_id
      ORDER BY s.order_date DESC, s.rowid ASC
    ) AS rn
  FROM stg_orders s
)
SELECT
  row_id, order_id, order_date, ship_date, ship_mode, customer_id, customer_name,
  segment, country, city, state, postal_code, region, retail_sales_people,
  product_id, category, sub_category, product_name, returned, sales, quantity, discount, profit
FROM ranked
WHERE rn = 1;

CREATE INDEX IF NOT EXISTS idx_dedup_order_date ON stg_orders_dedup(order_date);
CREATE INDEX IF NOT EXISTS idx_dedup_region_state ON stg_orders_dedup(region, state);

-- 2) Normalization view (text trimming, casing, returned standardization, numeric sanitization)
DROP VIEW IF EXISTS clean_orders;
CREATE VIEW clean_orders AS
WITH base AS (
  SELECT
    CAST(row_id AS INTEGER)                  AS row_id,
    TRIM(order_id)                           AS order_id,
    TRIM(order_date)                         AS order_date,   -- ISO TEXT yyyy-mm-dd
    TRIM(ship_date)                          AS ship_date,    -- ISO TEXT yyyy-mm-dd
    TRIM(ship_mode)                          AS ship_mode,
    TRIM(customer_id)                        AS customer_id,
    TRIM(segment)                            AS segment,
    UPPER(TRIM(country))                     AS country,
    TRIM(city)                               AS city,
    TRIM(state)                              AS state,
    CAST(postal_code AS INTEGER)             AS postal_code,
    UPPER(TRIM(region))                      AS region,
    TRIM(product_id)                         AS product_id,
    UPPER(TRIM(category))                    AS category,
    UPPER(TRIM(sub_category))                AS sub_category,
    TRIM(product_name)                       AS product_name,
    CASE
      WHEN returned IS NULL OR TRIM(returned) = '' THEN 'NO'
      WHEN UPPER(TRIM(returned)) IN ('Y','YES','TRUE','T','1','RETURNED') THEN 'YES'
      WHEN UPPER(TRIM(returned)) IN ('N','NO','FALSE','F','0','NOT RETURNED','NONE','NOT') THEN 'NO'
      ELSE UPPER(TRIM(returned))
    END                                      AS returned,
    CAST(sales    AS REAL)                   AS sales,
    CASE WHEN CAST(quantity AS REAL) < 0 THEN 0 ELSE CAST(quantity AS INTEGER) END AS quantity,
    CASE
      WHEN discount IS NULL THEN NULL
      WHEN discount > 1.0 AND discount <= 100.0 THEN ROUND(discount/100.0, 4)
      WHEN discount < 0.0 THEN 0.0
      ELSE CAST(discount AS REAL)
    END                                      AS discount,
    CAST(profit   AS REAL)                   AS profit
  FROM stg_orders_dedup
)
SELECT
  row_id, order_id, order_date, ship_date, ship_mode, customer_id, segment, country,
  city, state, postal_code, region, product_id, category, sub_category, product_name,
  returned, sales, quantity, discount, profit
FROM base;

-- 3) Materialize for BI consumption as Rsc_cleaned (PII excluded)
DROP TABLE IF EXISTS Rsc_cleaned;
CREATE TABLE Rsc_cleaned AS
SELECT * FROM clean_orders;

-- 4) Indexes on Rsc_cleaned
CREATE UNIQUE INDEX IF NOT EXISTS ux_rsc_row_id ON Rsc_cleaned(row_id);
CREATE INDEX IF NOT EXISTS idx_rsc_order_date ON Rsc_cleaned(order_date);
CREATE INDEX IF NOT EXISTS idx_rsc_region_state ON Rsc_cleaned(region, state);
CREATE INDEX IF NOT EXISTS idx_rsc_category_sub ON Rsc_cleaned(category, sub_category);
"""

# Execute cleaning
with sqlite3.connect(db_path.as_posix()) as conn:
    conn.executescript(cleaning_sql)
print("OK: cleaning executed -> table 'Rsc_cleaned' created/refreshed.")

# QA checks (concise)
with sqlite3.connect(db_path.as_posix()) as conn:
    cur = conn.cursor()
    counts = cur.execute("""
        SELECT
          (SELECT COUNT(*) FROM stg_orders),
          (SELECT COUNT(*) FROM stg_orders_dedup),
          (SELECT COUNT(*) FROM Rsc_cleaned)
    """).fetchone()
    uniq = cur.execute("""
        SELECT COUNT(*) AS total, COUNT(DISTINCT row_id) AS distinct_row_id
        FROM Rsc_cleaned
    """).fetchone()
    returned = cur.execute("""
        SELECT returned, COUNT(*) FROM Rsc_cleaned
        GROUP BY returned ORDER BY 2 DESC
    """).fetchall()
    num_sanity = cur.execute("""
        SELECT
          SUM(CASE WHEN discount < 0 OR discount > 1 THEN 1 ELSE 0 END),
          SUM(CASE WHEN quantity < 0 THEN 1 ELSE 0 END)
        FROM Rsc_cleaned
    """).fetchone()
    date_range = cur.execute("""
        SELECT MIN(order_date), MAX(order_date) FROM Rsc_cleaned
    """).fetchone()

print("Counts (stg_raw, stg_dedup, rsc_cleaned):", counts)
print("Row ID uniqueness (total, distinct):", uniq)
print("Returned distribution:", returned)
print("Numeric sanity (bad_discount, bad_qty):", num_sanity)
print("Order date range (min, max):", date_range)


OK: cleaning executed -> table 'Rsc_cleaned' created/refreshed.
Counts (stg_raw, stg_dedup, rsc_cleaned): (9994, 9994, 9994)
Row ID uniqueness (total, distinct): (9994, 9994)
Returned distribution: [('NO', 9194), ('YES', 800)]
Numeric sanity (bad_discount, bad_qty): (0, 0)
Order date range (min, max): ('2014-01-02', '2017-12-30')


In [6]:
# Fix viz_* creation: move CREATE TABLE before the WITH clause in section (2)

# Resolve DB path (supports project root or ./notebook)
cwd = Path.cwd()
data_dir_candidates = [cwd / "data", cwd.parent / "data"]
data_dir = next((p for p in data_dir_candidates if p.exists()), data_dir_candidates[0])
db_path = data_dir / "Retail_supply_chain.db"

viz_sql = """
-- Safety drops
DROP TABLE IF EXISTS viz_sales_by_region_month;
DROP TABLE IF EXISTS viz_top_category_by_region;
DROP TABLE IF EXISTS viz_returns_by_cat_subcat;

-- 1) Monthly sales by region (for interactive map + trend)
CREATE TABLE viz_sales_by_region_month AS
WITH d AS (
  SELECT date_key, year, month, year_month FROM dim_date
)
SELECT
  fs.region,
  fs.state,
  d.year,
  d.month,
  d.year_month,
  SUM(fs.sales)      AS sales,
  SUM(fs.profit)     AS profit,
  SUM(fs.quantity)   AS quantity,
  AVG(fs.discount)   AS avg_discount
FROM fct_sales fs
JOIN d ON fs.date_key = d.date_key
GROUP BY fs.region, fs.state, d.year, d.month, d.year_month;

CREATE INDEX IF NOT EXISTS idx_viz1_region_state_month ON viz_sales_by_region_month(region, state, year_month);

-- 2) Top-selling category by region (by total sales)
CREATE TABLE viz_top_category_by_region AS
WITH cat_region AS (
  SELECT
    fs.region,
    fs.category,
    SUM(fs.sales) AS sales
  FROM fct_sales fs
  GROUP BY fs.region, fs.category
),
ranked AS (
  SELECT
    cr.*,
    ROW_NUMBER() OVER (PARTITION BY cr.region ORDER BY cr.sales DESC) AS rn
  FROM cat_region cr
)
SELECT region, category, sales
FROM ranked
WHERE rn = 1;

CREATE INDEX IF NOT EXISTS idx_viz2_region ON viz_top_category_by_region(region);

-- 3) Returns by category/sub-category (count + rate)
CREATE TABLE viz_returns_by_cat_subcat AS
WITH base AS (
  SELECT
    UPPER(TRIM(category))      AS category,
    UPPER(TRIM(sub_category))  AS sub_category,
    COUNT(*)                   AS total_orders,
    SUM(CASE WHEN returned = 'YES' THEN 1 ELSE 0 END) AS returned_orders
  FROM fct_sales
  GROUP BY UPPER(TRIM(category)), UPPER(TRIM(sub_category))
)
SELECT
  category,
  sub_category,
  returned_orders,
  total_orders,
  CASE WHEN total_orders = 0 THEN 0.0
       ELSE ROUND(1.0 * returned_orders / total_orders, 4) END AS return_rate
FROM base;

CREATE INDEX IF NOT EXISTS idx_viz3_cat_sub ON viz_returns_by_cat_subcat(category, sub_category);
"""

with sqlite3.connect(db_path.as_posix()) as conn:
    conn.executescript(viz_sql)
print("OK: viz tables created (viz_sales_by_region_month, viz_top_category_by_region, viz_returns_by_cat_subcat).")

# Concise QA
with sqlite3.connect(db_path.as_posix()) as conn:
    cur = conn.cursor()
    sizes = cur.execute("""
        SELECT
          (SELECT COUNT(*) FROM viz_sales_by_region_month),
          (SELECT COUNT(*) FROM viz_top_category_by_region),
          (SELECT COUNT(*) FROM viz_returns_by_cat_subcat)
    """).fetchone()
    top_per_region = cur.execute("""
        SELECT region, COUNT(*) AS rows_per_region
        FROM viz_top_category_by_region
        GROUP BY region
    """).fetchall()
    return_rates_ok = cur.execute("""
        SELECT 
          SUM(CASE WHEN return_rate < 0 OR return_rate > 1 THEN 1 ELSE 0 END)
        FROM viz_returns_by_cat_subcat
    """).fetchone()[0]

print("Row counts (viz1, viz2, viz3):", sizes)
print("Rows per region in viz_top_category_by_region:", top_per_region)
print("Return_rate out-of-range rows (should be 0):", return_rates_ok)



OK: viz tables created (viz_sales_by_region_month, viz_top_category_by_region, viz_returns_by_cat_subcat).
Row counts (viz1, viz2, viz3): (1301, 4, 17)
Rows per region in viz_top_category_by_region: [('CENTRAL', 1), ('EAST', 1), ('SOUTH', 1), ('WEST', 1)]
Return_rate out-of-range rows (should be 0): 0


In [7]:
# Fix viz_* creation: move CREATE TABLE before the WITH clause in section (2)

# Resolve DB path (supports project root or ./notebook)
cwd = Path.cwd()
data_dir_candidates = [cwd / "data", cwd.parent / "data"]
data_dir = next((p for p in data_dir_candidates if p.exists()), data_dir_candidates[0])
db_path = data_dir / "Retail_supply_chain.db"

viz_sql = """
-- Safety drops
DROP TABLE IF EXISTS viz_sales_by_region_month;
DROP TABLE IF EXISTS viz_top_category_by_region;
DROP TABLE IF EXISTS viz_returns_by_cat_subcat;

-- 1) Monthly sales by region (for interactive map + trend)
CREATE TABLE viz_sales_by_region_month AS
WITH d AS (
  SELECT date_key, year, month, year_month FROM dim_date
)
SELECT
  fs.region,
  fs.state,
  d.year,
  d.month,
  d.year_month,
  SUM(fs.sales)      AS sales,
  SUM(fs.profit)     AS profit,
  SUM(fs.quantity)   AS quantity,
  AVG(fs.discount)   AS avg_discount
FROM fct_sales fs
JOIN d ON fs.date_key = d.date_key
GROUP BY fs.region, fs.state, d.year, d.month, d.year_month;

CREATE INDEX IF NOT EXISTS idx_viz1_region_state_month ON viz_sales_by_region_month(region, state, year_month);

-- 2) Top-selling category by region (by total sales)
CREATE TABLE viz_top_category_by_region AS
WITH cat_region AS (
  SELECT
    fs.region,
    fs.category,
    SUM(fs.sales) AS sales
  FROM fct_sales fs
  GROUP BY fs.region, fs.category
),
ranked AS (
  SELECT
    cr.*,
    ROW_NUMBER() OVER (PARTITION BY cr.region ORDER BY cr.sales DESC) AS rn
  FROM cat_region cr
)
SELECT region, category, sales
FROM ranked
WHERE rn = 1;

CREATE INDEX IF NOT EXISTS idx_viz2_region ON viz_top_category_by_region(region);

-- 3) Returns by category/sub-category (count + rate)
CREATE TABLE viz_returns_by_cat_subcat AS
WITH base AS (
  SELECT
    UPPER(TRIM(category))      AS category,
    UPPER(TRIM(sub_category))  AS sub_category,
    COUNT(*)                   AS total_orders,
    SUM(CASE WHEN returned = 'YES' THEN 1 ELSE 0 END) AS returned_orders
  FROM fct_sales
  GROUP BY UPPER(TRIM(category)), UPPER(TRIM(sub_category))
)
SELECT
  category,
  sub_category,
  returned_orders,
  total_orders,
  CASE WHEN total_orders = 0 THEN 0.0
       ELSE ROUND(1.0 * returned_orders / total_orders, 4) END AS return_rate
FROM base;

CREATE INDEX IF NOT EXISTS idx_viz3_cat_sub ON viz_returns_by_cat_subcat(category, sub_category);
"""

with sqlite3.connect(db_path.as_posix()) as conn:
    conn.executescript(viz_sql)
print("OK: viz tables created (viz_sales_by_region_month, viz_top_category_by_region, viz_returns_by_cat_subcat).")

# Concise QA
with sqlite3.connect(db_path.as_posix()) as conn:
    cur = conn.cursor()
    sizes = cur.execute("""
        SELECT
          (SELECT COUNT(*) FROM viz_sales_by_region_month),
          (SELECT COUNT(*) FROM viz_top_category_by_region),
          (SELECT COUNT(*) FROM viz_returns_by_cat_subcat)
    """).fetchone()
    top_per_region = cur.execute("""
        SELECT region, COUNT(*) AS rows_per_region
        FROM viz_top_category_by_region
        GROUP BY region
    """).fetchall()
    return_rates_ok = cur.execute("""
        SELECT 
          SUM(CASE WHEN return_rate < 0 OR return_rate > 1 THEN 1 ELSE 0 END)
        FROM viz_returns_by_cat_subcat
    """).fetchone()[0]

print("Row counts (viz1, viz2, viz3):", sizes)
print("Rows per region in viz_top_category_by_region:", top_per_region)
print("Return_rate out-of-range rows (should be 0):", return_rates_ok)



OK: viz tables created (viz_sales_by_region_month, viz_top_category_by_region, viz_returns_by_cat_subcat).
Row counts (viz1, viz2, viz3): (1301, 4, 17)
Rows per region in viz_top_category_by_region: [('CENTRAL', 1), ('EAST', 1), ('SOUTH', 1), ('WEST', 1)]
Return_rate out-of-range rows (should be 0): 0


In [9]:
# Export every TABLE from Retail_supply_chain.db to ./data/exports as CSV (no pandas, safe relative prints)

# Resolve project/data paths
cwd = Path.cwd()
project_root = cwd if (cwd / "data").exists() else cwd.parent
data_dir = project_root / "data"
db_path = data_dir / "Retail_supply_chain.db"
export_dir = data_dir / "exports"
export_dir.mkdir(parents=True, exist_ok=True)

def quote_ident(name: str) -> str:
    return '"' + name.replace('"', '""') + '"'

with sqlite3.connect(db_path.as_posix(), timeout=5) as conn:
    conn.executescript("""
        PRAGMA read_uncommitted=ON;
        PRAGMA synchronous=OFF;
        PRAGMA journal_mode=OFF;
        PRAGMA temp_store=MEMORY;
        PRAGMA cache_size=-200000;
    """)

    # List only user tables (exclude SQLite internals and views)
    tables = [r[0] for r in conn.execute(
        "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%' ORDER BY name;"
    )]

    for t in tables:
        # Ordered columns from PRAGMA (stable header)
        cols = [row[1] for row in conn.execute(f"PRAGMA table_info({quote_ident(t)});")]
        sql  = f"SELECT {', '.join(quote_ident(c) for c in cols)} FROM {quote_ident(t)};"

        out_path = export_dir / f"{t}.csv"
        with open(out_path, "w", encoding="utf-8", newline="", buffering=1_048_576) as f:
            w = csv.writer(f)
            w.writerow(cols)
            for row in conn.execute(sql):
                w.writerow(row)

        print(f"Exported: {(out_path.relative_to(project_root)).as_posix()}")

print("Done.")


Exported: data/exports/Rsc_cleaned.csv
Exported: data/exports/fct_sales.csv
Exported: data/exports/stg_orders.csv
Exported: data/exports/stg_orders_dedup.csv
Exported: data/exports/viz_returns_by_cat_subcat.csv
Exported: data/exports/viz_sales_by_region_month.csv
Exported: data/exports/viz_top_category_by_region.csv
Done.
